In [57]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    search_depth="advanced",  # basic or advanced
    # include_domains = []
    # exclude_domains = []
)

In [3]:
search_tool.invoke("포천 맛집")

[{'url': 'https://tour.click1-tip.com/entry/포천-맛집-베스트10',
  'content': '욕쟁이 할머니집 네이버 플레이스(구글 평점 3.7)\n맛집 소개\n이어지는 포천 맛집은 고모저수지 근처에 위치하고 있는 시래기 정식 전문점 욕쟁이 할머니집입니다. 나 혼자 산다와 찾아라 맛있는 TV 방송에 소개되었던 포천 고모리 맛집으로 일반 흙집을 개조하여 만든 아늑한 시골집 분위기의 한식당인데요. 된장과 두부에 우거지를 주재료로 한 우거지 정식에 청국장과 콩비지 그리고 숯불에 구운 불고기까지 가정식 백반처럼 드실 수 있는 포천 현지인 로컬 맛집입니다.\n메뉴 소개\n\n시래기정식 10,000원\n참숯불고기 한접시 12,000원\n맷돌두부 한접시 10,000원\n들기름두부 한접시 14,000원\n\n\n욕쟁이 할머니집 메뉴 이미지\n**\n매장 위치**\n3. 동이손만두(만두전골 맛집) [...] 포천 고모리691 네이버 플레이스(구글 평점 3.9)\n맛집 소개\n이어지는 포천 맛집은 고모리 카페마을 근처에 위치하고 있는 브런치 카페 고모리 691입니다. 10년 동안 블루리본 서베이에 선정된 포천 브런치 맛집으로 넓고 아늑한 분위기에 고모리 저수지를 보면서 파스타와 스테이크를 드시기 좋은 포천 고모리 카페인데요. 시그니처 메뉴인 누룽지 파스타에 스페셜 브런치와 스테이크까지 가족이나 연인과 함께 식사하기 좋은 포천 분위기 좋은 카페입니다.\n메뉴 소개\n\n스페셜 브런치 22,000원\n누룽지 파스타 22,000원\n돈마호크 스테이크 25,500원\n갈릭 오일 파스타 21,000원\n\n\n고모리691 주요 이미지\n매장 위치\n5. 양문한식부페(가성비 한식뷔페) [...] 효담곤드레산채밥상 네이버 플레이스(구글 평점 3.6)\n맛집 소개\n이어지는 포천 맛집은 광릉수목원 근처에 위치하고 있는 산나물 밥성 전문 한식당 고모리 효담곤드레 산채밥상입니다. 생방송투데이와 생방송오늘저녁, 생생정보 TV 방송에 여러 차례 소개되었던 포천 밥집

In [4]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


class Info(BaseModel):
    name: str = Field(..., description="식당 이름")
    address: str = Field(..., description="식당 주소")
    subway: str = Field(..., description="식당 지하철역")
    lat: str = Field(..., description="식당 위도")
    lng: str = Field(..., description="식당 경도")
    menu: str = Field(..., description="식당 메뉴")
    review: str = Field(..., description="식당 후기")
    url: str = Field(..., description="참고한 url")


class Answer(BaseModel):
    answer: str = Field(..., description="답변 내용")
    infos: list[Info] = Field(..., description="맛집 정보 리스트")


parser = JsonOutputParser(pydantic_object=Answer)

In [41]:
# 템플릿 수정: search_results를 제거하거나 처리 방법 변경
TEMPLATE = """당신은 맛집을 찾아서 정리하는 블로거 입니다.

사용자 질문에 따른 맛집 정보를 search_tool을 사용하여 검색 후 정리하여 식당 데이터를 정리 합니다.

그 후 확인 된 식당 이름을 기반으로 get_restraunt_info 도구를 사용하여 주소 및 좌표를 추출해주세요.
단, 식당이름으로 조회가 되지 않을 시 식당이름의 오탈자를 조정하여 다시 조회해주세요.

추출된 좌표를 기반으로 get_subway_info 도구를 사용하여 지하철역 정보를 추출해주세요.

사용자 질문:
{input}

출력 형식:

{{
    "answer": "간단하게 정리된 답변",
    "infos": [
        {{
            "name": "식당 이름",
            "address": "식당 주소",
            "subway": "식당 지하철역",
            "lat": "식당 위도",
            "lng": "식당 경도",
            "menu": "메뉴1, 메뉴2, ...",
            "review": "식당 후기",
            "url": "참고한 url"
        }},
        {{
            "name": "식당 이름2",
            "address": "식당 주소2",
            "subway": "식당 지하철역2",
            "lat": "식당 위도2",
            "lng": "식당 경도2",
            "menu": "메뉴1, 메뉴2, ...",
            "review": "식당 후기2",
            "url": "참고한 url2"
        }},
        ...
    ]
}}
{agent_scratchpad}
"""

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests
import os

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)

# LLM 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)


# search_tool을 LangChain 형식에 맞게 변환
@tool
def search(query: str) -> list:
    """Search for information about restaurants and food."""
    return search_tool.invoke(query)


@tool
def get_restraunt_info(restraunt_name: str) -> list:
    """Get the coordinates of a restaurant."""
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {"query": restraunt_name}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["documents"]:
            address = f"{data['documents'][0]['address_name']} (도로명 : {data['documents'][0]['road_address_name']})"
            latitude = data["documents"][0]["y"]
            longitude = data["documents"][0]["x"]
            return address, latitude, longitude
        else:
            return "정보 없음", "정보 없음", "정보 없음"


@tool
def get_subway_info(latitude: str, longitude: str) -> list:
    """Get the subway information of a restaurant."""
    station_url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {os.getenv('KAKAO_API_KEY')}"}
    params = {
        "category_group_code": "SW8",
        "x": longitude,
        "y": latitude,
        "radius": 2000,
        "sort": "distance",
    }
    response = requests.get(station_url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["documents"]:
            station_name = data["documents"][0]["place_name"]
            station_distance = data["documents"][0]["distance"]
            return station_name, station_distance
        else:
            return "정보 없음", "정보 없음"

In [46]:
tools = [search, get_restraunt_info, get_subway_info]

agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [47]:
formatting_prompt = ChatPromptTemplate.from_template(
    """다음 정보를 정확한 JSON 형식으로 변환해주세요:
    {agent_result}
    
    JSON 형식:
    {format_instructions}
    """
)

In [48]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"input": RunnablePassthrough()}
    | agent_executor
    | (
        lambda x: {
            "agent_result": x["output"],
            "format_instructions": parser.get_format_instructions(),
        }
    )
    | formatting_prompt
    | llm
    | parser
)

In [49]:
chain.invoke({"input": "논현역 맛집에 대해서 많이 찾아줘"})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '논현역 맛집'}`


[{'url': 'https://hotel-iu.tistory.com/1970', 'content': "논현역 맛집 베스트 추천 top 10논현역 맛집 베스트 추천 top 10곳을 소개합니다. 1: 우기식당상호명: 우기식당주소: 서울특별시 강남구 논현동 144-8전화번호: 미입력관련 키워드: 키워드 미입력관련 태그: ['데이트하기 좋은', '젊고 캐쥬얼한', '모임하기 좋은']영업시간: 평일 17:00~03:00 평일 (월. 화. 수"}, {'url': 'https://m.blog.naver.com/jjjshong7/223710911528', 'content': '기름진 맛을 깔끔하게 잡아주는 개운한 칼국수는 시원한\n\n감칠맛을 자랑하고 있었어요.\n\n\u200b\n\n서울특별시 서초구 강남대로89길 14\n\n\u200b\n\n4.논현역 맛집 도셰프 파스타&디아볼라피자\n\n주소:서울 서초구 강남대로89길 14\n\n번호: 0507-1433-1136\n\n영업시간: 11:30 - 22:00\n\n주차: 전용주차장 가능\n\n\u200b\n\n마지막으로 찾아간 곳은 도셰프로 이탈리아 음식을 파는\n\n곳인데 무려 서울 5대 화덕피자로도 알려져 있었답니다.\n\n논현역 4번출구 기준으로 도보로 1분거리에 위치하고 있던\n\n곳으로 식사가 가능한 시간은 매일 오전 11시30분부터 오후\n\n22시까지로 브레이크타임은 15시부터 17시까지였기에\n\n맛있는녀석들 106회에도 소개된적이 있었던 곳으로 쭉\n\n둘러보고 마음에 드는 곳에 자리를 잡은 다음에 디아볼라\n\n그리고 파스타까지 주문을 해 봤어요. 모짜렐라 치즈와\n\n스파이시 살라미, 크러쉬드레드페퍼와 토마토소스가 잘\n\n어우러진 피자는 직접 반죽한 도우의 쫀득함과 화덕에서 [...] 식사가 가능한 시간은 매일 오후 16시30분부터 22시30분\

{'answer': '논현역 주변의 다양한 맛집을 소개합니다. 각 식당은 독특한 메뉴와 분위기를 자랑하며, 지하철역과의 접근성도 뛰어납니다.',
 'infos': [{'name': '우기식당',
   'address': '서울 중구 을지로3가 302-7 (도로명 : 서울 중구 충무로5길 18)',
   'subway': '을지로3가역 2호선',
   'lat': '37.5653518187627',
   'lng': '126.991804032228',
   'menu': '젊고 캐쥬얼한 모임하기 좋은 장소',
   'review': '데이트하기 좋은 분위기와 모임하기 좋은 장소로 추천됩니다.',
   'url': 'https://hotel-iu.tistory.com/1970'},
  {'name': '도셰프 파스타&디아볼라피자',
   'address': '서울 서초구 반포동 707-8 (도로명 : 서울 서초구 강남대로89길 14)',
   'subway': '논현역 신분당선',
   'lat': '37.5094719731264',
   'lng': '127.021114237763',
   'menu': '디아볼라 피자, 파스타',
   'review': '서울 5대 화덕피자로 알려진 이탈리안 레스토랑입니다.',
   'url': 'https://m.blog.naver.com/jjjshong7/223710911528'},
  {'name': '진미평양냉면',
   'address': '서울 강남구 논현동 115-10 (도로명 : 서울 강남구 학동로 305-3)',
   'subway': '학동역 7호선',
   'lat': '37.5161357904841',
   'lng': '127.036047158128',
   'menu': '평양냉면',
   'review': '깔끔하고 오리지널의 평양냉면을 맛볼 수 있는 곳입니다.',
   'url': 'https://m.blog.naver.com/jjjshong7/223710911528'},
  {'name': '호루

In [16]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

NAVER_CLIENT_ID = os.getenv('NAVER_CLIENT_ID')
NAVER_CLIENT_SECRET = os.getenv('NAVER_CLIENT_SECRET')

def get_restraunt_info(restraunt_name: str):
    url = "https://openapi.naver.com/v1/search/local.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    }
    params = {"query": restraunt_name, "display": 1}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["items"]:
            result = data["items"][0]
            address = result.get("address", "정보 없음")
            mapx = int(result.get("mapx")) / 1e7
            mapy = int(result.get("mapy")) / 1e7
            return address, mapy, mapx
        else:
            return "정보 없음", "정보 없음", "정보 없음"


def get_subway_info(latitude: float, longitude: float):
    url = "https://openapi.naver.com/v1/search/local.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
    }
    params = {
        "query": "지하철역",
        "display": 1,
        "start": 1,
        "sort": "distance",
        "longitude": longitude,
        "latitude": latitude,
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["items"]:
            station = data["items"][0]
            station_name = station.get("title", "정보 없음").replace('<b>', '').replace('</b>', '')
            station_distance = station.get("distance", "정보 없음")
            return station_name, f"{station_distance}m"
        else:
            return "정보 없음", "정보 없음"


In [17]:
url = "https://openapi.naver.com/v1/search/local.json"
headers = {
    "X-Naver-Client-Id": NAVER_CLIENT_ID,
    "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
}
params = {"query": "우원돈가스", "display": 1}
response = requests.get(url, headers=headers, params=params)